In [ ]:
"""
@author: abaglione and lihuacai

Credit to Tyler Spears and Sonia Baee, who developed the precursor
to this preprocessing script
"""

# imports
import sys
import os
import functools
import pathlib
import glob
import collections
import itertools
import re
import random
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

import numpy as np
import pandas as pd
import copy
import pipeline

from sklearn import impute
from sklearn import datasets
from sklearn import svm, linear_model, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import scipy
from scipy.spatial.distance import cdist

# visualization libraries
%matplotlib inline
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.autolayout': True})
plt.rcParams.update({'figure.facecolor': [1.0, 1.0, 1.0, 1.0]})

# configure autoreloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Read in the weekly feature vectors 
all_feats = pd.read_csv('features/app_users_only/all_ind_wkly.csv')
all_feats.drop(axis=1,columns=['Unnamed: 0'] + \
               [col for col in all_feats.columns if 'trait' in col and 'group' in col],
               inplace=True)
all_feats['pid'] = all_feats['pid'].astype(str)

# Store the feature names
featnames = list(all_feats.columns)

all_feats

In [ ]:
all_feats

In [ ]:
#outcomes transformation -- anx, dep
#week to week change as outcome
#change to baseline level as outcome
#instead of difference, consider ratio between the weekly value and the baseline
#global average being subtracted

all_feats['anx'].hist()
all_feats['dep'].hist()

#add classification outcomes
# TODO: only add this after anx and dep are imputed!
# TODO: Make groups match Lee's interpretation here?
# TODO: add lee's reversal of depression scale back in
all_feats['dep_cat'] = np.where(all_feats['dep'] >= 4, 1, 0)
all_feats['anx_cat'] = np.where(all_feats['anx'] >= 3, 1, 0)

In [ ]:
# Create a subset with survey features + only features from the most used app
mua_dfs = []
most_used_app_cols = [col for col in all_feats.columns if 'most_used_app' in col]
    
for i in range(all_feats.shape[0]):
    
    # Get the current row as a dataframe
    df = all_feats.iloc[[i]]

    # Find the most used apps - retain only the first one
    df2 = df[most_used_app_cols]
    most_used_apps = list(df2.columns[(df2 == 1).any(axis=0)])
    most_used_app = [col.replace('most_used_app_', '') for col in most_used_apps][0]

    # Get a list of columns that should be included in the final df
    app_cols = [col for col in app_ind_fs_cols if most_used_app in col]
    df2 = df[app_cols]
    df2.columns = df2.columns.str.replace('_' + most_used_app, '')
    mua_dfs.append(df2)
    
app_mua_feats = pd.concat(mua_dfs, sort=False)
app_mua_feats

In [ ]:
survey_app_mua_feats = pd.concat(
    [all_feats[['pid', 'weekofstudy', 'anx', 'dep','anx_cat', 'dep_cat']],
     all_feats[survey_fs_cols],
     app_mua_feats], 
    axis=1, copy=True
)

survey_app_mua_feats

In [ ]:
# ------ Feature Set Spec -----------

APPS = ['aspire', 'boostme', 'dailyfeats', 'icope', 'mantra', 'messages',
        'moveme', 'relax', 'slumbertime', 'thoughtchallenger', 'worryknot']
ENGAGEMENT_METRICS = ['frequency', 'duration',
                      'betweenlaunch_duration', 'days_of_use']
TIMES_OF_DAY = ['morning', 'afternoon', 'evening', 'late_night']
TIMEDIV_COL = 'week_of_study'

# Survey Features Only
survey_fs_cols = ['cope_alcohol_tob', 'physical_pain', 'connected', 'receive_support', 'active',
                  'support_others', 'healthy_food']

# App Features - Aggregate, Across All Apps
app_overall_fs_cols = [TIMEDIV_COL, 'frequency', 'daysofuse', 'duration', 'duration_mean',
                       'duration_std', 'duration_min', 'duration_max', 'betweenlaunch_duration_mean',
                       'betweenlaunch_duration_std', 'num_apps_used'] + mua_dummy_cols

# App Features - From Only Most Used App, for Each Observation
app_mua_fs_cols = [TIMEDIV_COL] + \
                  [col for col in app_overall_fs_cols if 'num_apps_used' not in col] + \
                  mua_dummy_cols

# App Features - From Individual Apps
app_ind_fs_cols = [TIMEDIV_COL] + \
    [col for col in all_feats.columns
     if any([app in col for app in APPS])
     and any([agg_metric in col for agg_metric in ENGAGEMENT_METRICS])
     and not any([tod in col for tod in TIMES_OF_DAY])]

app_ind_fs_cols[0:5]

In [ ]:
# Create dictionary of featuresets
featuresets = {
    'survey_fs': survey_fs_cols,
    'app_overall_fs': app_overall_fs_cols,
    'app_ind_fs': app_ind_fs_cols,
    'app_mua_fs': app_mua_fs_cols,
    'survey_app_overall_fs': survey_fs_cols+app_overall_fs_cols, 
    'survey_app_ind_fs': survey_fs_cols+app_ind_fs_cols,
    'survey_app_mua_fs': survey_fs_cols+app_mua_fs_cols
}

In [ ]:
featuresets

In [ ]:
# Drop columns of all NaNs
all_feats.dropna(axis=1, how='all', inplace=True)
survey_app_mua_feats.dropna(axis=1, how='all', inplace=True)

In [ ]:
all_feats['anx']

In [ ]:
######regression tasks on 1-5 scale (cut off on both 1 (floor) and 5 (ceiling)) using lasso linear mixed effect model;
alpha_list = np.arange(0.1, 0.81, 0.1)
lmm_res = []

for alpha in alpha_list:
    print('alpha: {0}'.format(alpha))
    for fs_name, fs_cols in featuresets.items():
        if 'mua' in fs_name:
            df = survey_app_mua_feats.copy()
        else:
            df = all_feats.copy()
        
        df['intercept'] = 1
        df.to_csv('features/%s.csv' % fs_name)
        
        for target in ['anx', 'dep']:
            res = pipeline.genMixedLM(df, target, ['intercept'] + fs_cols,
                            'pid', fs_name, alpha=alpha)
            lmm_res.append(res.copy())

lmm_res = pd.concat(lmm_res, copy=True, ignore_index=True, sort=False)
lmm_res.to_csv('results/lmm_res.csv', index=False)

In [ ]:
# Prediction
id_col = 'pid'
target_cols = ['anx_cat', 'dep_cat']

for fs_name, fs_cols in featuresets.items():

    if 'mua' not in fs_name:
        df = all_feats
    else:
        # Handle special cases in which we want data only from the most used app
        df = survey_app_mua_feats

    X = df[[id_col] + fs_cols].copy()
    
    ''' If this is a featureset with app features 
        Get a list of one-hot-encoded columns from the most_used_app feature.'''
    mua_onehots = [col for col in X.columns if 'most_used_app' in col]
    
    print(X.columns)
    # Get categorical feature indices - will be used with SMOTENC later
    nominal_idx = sorted([X.columns.get_loc(c) for c in ['pid'] + mua_onehots])

    # y
    targets = {
        'anxiety': df['anx_cat'].copy(),
        'depression': df['dep_cat'].copy()
    }

    for target_name, target_col in targets.items():
        for method in ['RF', 'XGB']:
            res = pipeline.classifyMood(X=X, y=target_col, id=id_col, target=target_name,
                              nominal_idx = nominal_idx, fs=fs_name, method=method)
